In [1]:
import json
import psycopg2
from psycopg2.extras import RealDictCursor
import os
from dotenv import load_dotenv
import pandas as pd
import numpy as np

load_dotenv()

True

In [2]:
# Fetch the values from environment variables
api_key = os.getenv("API_KEY")
db_user = os.getenv("DB_USER")
db_pass = os.getenv("DB_PASS")
db_host = os.getenv("DB_HOST")
db_port = os.getenv("DB_PORT")

conn = psycopg2.connect(
    dbname="box_scores",
    user=db_user,
    password=db_pass,
    host=db_host,
    port=db_port
)

In [3]:
query = """
WITH current_games AS (
    SELECT array_agg(game_id) AS game_ids
    FROM game
    WHERE game.date < CURRENT_DATE
)
SELECT pg.player_id, g.date, p.first_name, p.last_name, pg.fgm, pg.fga, pg.fg_pct, pg.fg3m, pg.fg3a, pg.fg3_pct, pg.ftm, pg.fta, pg.ft_pct, pg.oreb, pg.dreb, pg.reb, pg.ast, pg.stl, pg.blk, pg.turnover, pg.pf, pg.pts
FROM player_game pg
INNER JOIN player p ON pg.player_id=p.player_id
INNER JOIN game g ON pg.game_id=g.game_id
WHERE pg.game_id = ANY (SELECT unnest(game_ids) FROM current_games);
"""

In [4]:
df = pd.read_sql_query(query, conn)
conn.close()
df.to_json('data/preprocessed.json')

/var/folders/lt/n_z12_lx2cb6n9qf36pmd3x40000gn/T/ipykernel_78912/4076807213.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, conn)
